In [1]:
import os
import re

import pystac
import pystac.item
import pystac.link
from pystac.provider import ProviderRole
from pystac.extensions.projection import ProjectionExtension


In [8]:
WORKING_DIR = os.getcwd()
STAC_DIR = 'stac_tests'
COLLECTION_ID = 'corine-land-cover-raster'
COLLECTION_TITLE = "CORINE Land Cover Raster"
COLLECTION_DESCRIPTION = ("The European Commission launched the CORINE (Coordination of Information on the Environment) "
                          "program in an effort to develop a standardized methodology for producing continent-scale land "
                          "cover, biotope, and air quality maps. The CORINE Land Cover (CLC) product offers a pan-European "
                          "land cover and land use inventory with 44 thematic classes, ranging from broad forested areas "
                          "to individual vineyards.")



# CLMS_CATALOG_LINK = pystac.link.Link(
#     rel=pystac.RelType.ROOT, target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, "stacs/clms_catalog.json"))
# )
# COLLECTION_LINK = pystac.link.Link(
#     rel=pystac.RelType.COLLECTION,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )
# ITEM_PARENT_LINK = pystac.link.Link(
#     rel=pystac.RelType.PARENT,
#     target=pystac.STACObject.from_file(os.path.join(WORKING_DIR, f"stacs/{COLLECTION_ID}/{COLLECTION_ID}.json")),
# )

In [9]:

sp_extent = pystac.SpatialExtent([None, None, None, None])
tmp_extent = pystac.TemporalExtent(["1990-01-01T00:00:00.000Z", None])
extent = pystac.Extent(sp_extent,  tmp_extent)

collection = pystac.Collection(id=COLLECTION_ID,
                               description=COLLECTION_DESCRIPTION,
                               title=COLLECTION_TITLE,
                               extent=extent,
                               )